In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/delhi-neighborhood-data/delhi_dataSet.csv
/kaggle/input/delhi-neighborhood-data/restaurant_dataSet.csv


In [70]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize

# Data Collection
**Using Kaggle dataset** the data is collected.

In [71]:
Neighbourhood_df= pd.read_csv('../input/delhi-neighborhood-data/delhi_dataSet.csv')
Neighbourhood_df.head(100)

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
4,4,North West Delhi,Begum Pur,NaN,NaN
...,...,...,...,...,...
95,95,South Delhi,Alaknanda,28.529336,77.251632
96,96,South Delhi,Chhattarpur,28.507007,77.175417
97,97,South Delhi,Chittaranjan Park,28.538752,77.249249
98,98,South Delhi,Dayanand Colony,28.562200,77.247613


In [72]:
Neighbourhood_df.dropna( inplace=True)
Neighbourhood_df.reset_index(drop= True)
Neighbourhood_df.head()

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
5,5,North West Delhi,Dhaka,39.031714,-90.261223


In [73]:
Neighbourhood_df.head()

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
5,5,North West Delhi,Dhaka,39.031714,-90.261223


In [74]:
CLIENT_ID = 'XG4LJAR2GM4UAGSNIUJPKU2SQ5MVM2E2Z2CYQTOASVKRPNVZ'
CLIENT_SECRET = 'LSG2VCTFX0AIYR0Y20BMIB2UT4MPVXUOKEXFLZ0YQABCPC4O'
VERSION = '20180605'

print('MY CLIENT_ID: ' + CLIENT_ID)
print('MY CLIENT_SECRET:' + CLIENT_SECRET)

MY CLIENT_ID: XG4LJAR2GM4UAGSNIUJPKU2SQ5MVM2E2Z2CYQTOASVKRPNVZ
MY CLIENT_SECRET:LSG2VCTFX0AIYR0Y20BMIB2UT4MPVXUOKEXFLZ0YQABCPC4O


In [75]:
latitude = 28.7041
longitude =  77.1025
print('The geograpical coordinate of Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi are 28.7041, 77.1025.


# Folium Map Import For Delhi

In [76]:
import folium as fo
delhi_map = fo.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(Neighbourhood_df['latitude'], Neighbourhood_df['longitude'], Neighbourhood_df['Neighborhood']):
    label = fo.Popup(label, parse_html=True)
    fo.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#a45b2c',
        fill_opacity=0.7,
        parse_html=False).add_to(delhi_map)  
    
delhi_map

# Adding the Data for Venues Using Foursquare API

In [77]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
      
        results = requests.get(url).json()["response"]['groups'][0]['items']
      
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'latitude', 
                  'longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues) 

In [78]:
Delhi_Venues = getNearbyVenues(names=Neighbourhood_df['Neighborhood'],latitudes=Neighbourhood_df['latitude'],longitudes=Neighbourhood_df['longitude'])

In [79]:

print("Our venues dataset includes", Delhi_Venues.shape[0], 'venues, each has', Delhi_Venues.shape[1], 'attributes')
print('There are', len(Delhi_Venues['Venue Category'].unique()), 'categories in total')
Delhi_Venues.head(20)

Our venues dataset includes 1042 venues, each has 7 attributes
There are 165 categories in total


,Neighborhood,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614192,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Ashok Vihar,28.699453,77.184826,ELIXIR FERTILITY CENTRE,28.700034,77.188093,Health & Beauty Service
2,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery
3,Ashok Vihar,28.699453,77.184826,Invitation Banquet,28.696018,77.185953,Diner
4,Ashok Vihar,28.699453,77.184826,Gola Northend,28.701242,77.189288,Indian Restaurant
5,Azadpur,28.707657,77.175547,Azadpur,28.708669,77.179102,Bus Station
6,Azadpur,28.707657,77.175547,Cues N Ball,28.711260,77.176680,Pool Hall
7,Azadpur,28.707657,77.175547,Tulip Banquet,28.704523,77.172441,Restaurant
8,Bawana,28.799660,77.032885,Rajiv Gandhi Stadium,28.801811,77.028751,Playground
9,Jahangirpuri,28.725972,77.162658,Jahangirpuri Metro Station | जहांगीरपुरी,28.726026,77.162626,Light Rail Station


In [80]:
onehot = pd.get_dummies(Delhi_Venues[['Venue Category']], prefix="", prefix_sep="")
onehot.insert(0, 'Neighbourhood', Delhi_Venues['Neighborhood'])


In [81]:
grouped = onehot.groupby('Neighbourhood').sum().reset_index()
grouped.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alaknanda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Anand Vihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ashok Vihar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Azadpur,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [83]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighborhood'] = grouped['Neighbourhood']

for ind in np.arange(grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(95)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Indian Restaurant,Yoga Studio,Food,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
1,Alaknanda,Indian Restaurant,BBQ Joint,Pizza Place,Food & Drink Shop,New American Restaurant,Coffee Shop,Restaurant,Steakhouse,Middle Eastern Restaurant,Food
2,Anand Vihar,Indian Restaurant,Furniture / Home Store,Pizza Place,Indian Sweet Shop,Ice Cream Shop,Soup Place,Food,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
3,Ashok Vihar,Diner,Indian Restaurant,Health & Beauty Service,Bakery,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
4,Azadpur,Pool Hall,Restaurant,Bus Station,Yoga Studio,Food,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
...,...,...,...,...,...,...,...,...,...,...,...
90,Paharganj,Fast Food Restaurant,Platform,Train Station,Food & Drink Shop,Food,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
91,Pandav Nagar,Wings Joint,Indian Restaurant,Train Station,Arcade,Movie Theater,Yoga Studio,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop
92,Paschim Vihar,Jewelry Store,Garden,Coffee Shop,Park,BBQ Joint,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
93,Patel Nagar,Café,Yoga Studio,Gastropub,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court


In [84]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [85]:
!pip install sklearn

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


# Clustering using KMeans


In [86]:
Neighbourhood_df.dropna(inplace= True)
Neigh_df = Neighbourhood_df.reset_index(drop = True) 


In [87]:
delhi_df = pd.merge(Neigh_df, neighbourhoods_venues_sorted, on='Neighborhood', how='right')
delhi_df.dropna(inplace= True)
Neigh_df = delhi_df.reset_index(drop = True) 
Neigh_df.head(143)

,Unnamed: 0,Borough,Neighborhood,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541,Indian Restaurant,Yoga Studio,Food,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
1,95,South Delhi,Alaknanda,28.529336,77.251632,Indian Restaurant,BBQ Joint,Pizza Place,Food & Drink Shop,New American Restaurant,Coffee Shop,Restaurant,Steakhouse,Middle Eastern Restaurant,Food
2,87,East Delhi,Anand Vihar,28.625691,77.101941,Indian Restaurant,Furniture / Home Store,Pizza Place,Indian Sweet Shop,Ice Cream Shop,Soup Place,Food,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
3,1,North West Delhi,Ashok Vihar,28.699453,77.184826,Diner,Indian Restaurant,Health & Beauty Service,Bakery,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
4,2,North West Delhi,Azadpur,28.707657,77.175547,Pool Hall,Restaurant,Bus Station,Yoga Studio,Food,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,91,East Delhi,Vasundhara Enclave,28.601726,77.321122,Pizza Place,Convenience Store,Stadium,Market,Food,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
139,184,West Delhi,Vikaspuri,28.638419,77.070836,Pizza Place,Indian Restaurant,Flea Market,Yoga Studio,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
140,92,East Delhi,Vishwas Nagar,28.664470,77.291741,Cosmetics Shop,Electronics Store,Yoga Studio,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
141,93,East Delhi,Vivek Vihar,28.669164,77.312267,Juice Bar,Yoga Studio,Food & Drink Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck


In [88]:
kclusters = 5
X = grouped.drop('Neighbourhood', 1).astype(float)
X = StandardScaler().fit_transform(X)
kmeans= KMeans(n_clusters=kclusters, random_state=0).fit(X)
yhat = kmeans.labels_

In [89]:
delhi_clustering = grouped.drop('Neighbourhood', 1)
neighbourhoods_venues_sorted.insert(5, 'Cluster Label', yhat)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(delhi_clustering)

Delhi_merged = Delhi_Venues.join(neighbourhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
Delhi_merged.head()

,Neighborhood,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,Cluster Label,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,28.614192,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant,Indian Restaurant,Yoga Studio,Food,Garden,2,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
1,Ashok Vihar,28.699453,77.184826,ELIXIR FERTILITY CENTRE,28.700034,77.188093,Health & Beauty Service,Diner,Indian Restaurant,Health & Beauty Service,Bakery,2,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
2,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery,Diner,Indian Restaurant,Health & Beauty Service,Bakery,2,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
3,Ashok Vihar,28.699453,77.184826,Invitation Banquet,28.696018,77.185953,Diner,Diner,Indian Restaurant,Health & Beauty Service,Bakery,2,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
4,Ashok Vihar,28.699453,77.184826,Gola Northend,28.701242,77.189288,Indian Restaurant,Diner,Indian Restaurant,Health & Beauty Service,Bakery,2,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


In [92]:
map_clusters = fo.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Delhi_merged['latitude'],Delhi_merged['longitude'], Delhi_merged['Neighborhood'],Delhi_merged['Cluster Label']):
    label = fo.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fo.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [94]:
Delhi_merged[Delhi_merged['Cluster Label'] == 0]

,Neighborhood,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,Cluster Label,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
829,Sriniwaspuri,28.565220,77.254704,Srinivas Puri Xing,28.565577,77.252608,Plaza,Plaza,Wine Bar,Airport,Food,0,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
830,Sriniwaspuri,28.565220,77.254704,AIESEC Delhi IIT Office,28.562111,77.251912,Wine Bar,Plaza,Wine Bar,Airport,Food,0,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
831,Sriniwaspuri,28.565220,77.254704,emirates airport lounge delhi,28.560982,77.255481,Airport,Plaza,Wine Bar,Airport,Food,0,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
922,Vasant Vihar,28.560691,77.160791,Godrej's Nature's Basket,28.557431,77.163854,Gourmet Shop,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Indian Restaurant,0,Market,Department Store,Pizza Place,Nightlife Spot,Pub,Restaurant
923,Vasant Vihar,28.560691,77.160791,Modern Bazaar,28.558854,77.163898,Department Store,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Indian Restaurant,0,Market,Department Store,Pizza Place,Nightlife Spot,Pub,Restaurant
924,Vasant Vihar,28.560691,77.160791,Le Marche,28.557331,77.164001,Food & Drink Shop,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Indian Restaurant,0,Market,Department Store,Pizza Place,Nightlife Spot,Pub,Restaurant
925,Vasant Vihar,28.560691,77.160791,Sartoria,28.558937,77.164182,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Indian Restaurant,0,Market,Department Store,Pizza Place,Nightlife Spot,Pub,Restaurant
926,Vasant Vihar,28.560691,77.160791,Yo! China,28.557967,77.164270,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Indian Restaurant,0,Market,Department Store,Pizza Place,Nightlife Spot,Pub,Restaurant
927,Vasant Vihar,28.560691,77.160791,Turquoise Cottage,28.557387,77.163484,Pub,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Indian Restaurant,0,Market,Department Store,Pizza Place,Nightlife Spot,Pub,Restaurant
928,Vasant Vihar,28.560691,77.160791,KFC,28.557477,77.164089,Fast Food Restaurant,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Indian Restaurant,0,Market,Department Store,Pizza Place,Nightlife Spot,Pub,Restaurant


In [95]:
Delhi_merged[Delhi_merged['Cluster Label'] ==1]

,Neighborhood,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,Cluster Label,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
302,Khan Market,28.600135,77.226491,SODABOTTLEOPENERWALA,28.600141,77.226273,Irani Cafe,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store
303,Khan Market,28.600135,77.226491,Khan Market | खान मार्केट (Khan Market),28.600342,77.226923,Market,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store
304,Khan Market,28.600135,77.226491,The Big Chill Cafe,28.600686,77.227636,Italian Restaurant,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store
305,Khan Market,28.600135,77.226491,Perch,28.599987,77.226350,Café,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store
306,Khan Market,28.600135,77.226491,Public Affair,28.600030,77.226478,Lounge,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store
307,Khan Market,28.600135,77.226491,L'Opera,28.599784,77.226144,Café,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store
308,Khan Market,28.600135,77.226491,"Perch, Wine And Coffee Bar",28.600058,77.226406,Coffee Shop,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store
309,Khan Market,28.600135,77.226491,Smoke House Deli,28.599850,77.226900,French Restaurant,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store
310,Khan Market,28.600135,77.226491,Khan Chacha,28.600618,77.227237,Indian Restaurant,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store
311,Khan Market,28.600135,77.226491,Bahri Sons Booksellers,28.599977,77.227134,Bookstore,Café,Coffee Shop,Bar,Chinese Restaurant,1,Indian Restaurant,Asian Restaurant,Bookstore,Boutique,Sandwich Place,Furniture / Home Store


In [96]:
Delhi_merged[Delhi_merged['Cluster Label'] == 2]

,Neighborhood,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,Cluster Label,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,28.614192,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant,Indian Restaurant,Yoga Studio,Food,Garden,2,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
1,Ashok Vihar,28.699453,77.184826,ELIXIR FERTILITY CENTRE,28.700034,77.188093,Health & Beauty Service,Diner,Indian Restaurant,Health & Beauty Service,Bakery,2,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
2,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery,Diner,Indian Restaurant,Health & Beauty Service,Bakery,2,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
3,Ashok Vihar,28.699453,77.184826,Invitation Banquet,28.696018,77.185953,Diner,Diner,Indian Restaurant,Health & Beauty Service,Bakery,2,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
4,Ashok Vihar,28.699453,77.184826,Gola Northend,28.701242,77.189288,Indian Restaurant,Diner,Indian Restaurant,Health & Beauty Service,Bakery,2,Food Court,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,Tilak Nagar,28.639650,77.094039,sarna electronics,28.640043,77.089984,Electronics Store,Fast Food Restaurant,Women's Store,Electronics Store,Indian Restaurant,2,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
1038,Tilak Nagar,28.639650,77.094039,Aggarwal Restaurant,28.636747,77.097177,Indian Restaurant,Fast Food Restaurant,Women's Store,Electronics Store,Indian Restaurant,2,Food Court,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
1039,Vikaspuri,28.638419,77.070836,Domino's Pizza,28.638000,77.075000,Pizza Place,Pizza Place,Indian Restaurant,Flea Market,Yoga Studio,2,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
1040,Vikaspuri,28.638419,77.070836,Budella Market,28.636387,77.067748,Flea Market,Pizza Place,Indian Restaurant,Flea Market,Yoga Studio,2,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


In [100]:
Delhi_merged[Delhi_merged['Cluster Label'] == 3]

,Neighborhood,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,Cluster Label,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
532,Hauz Khas Village,28.553855,77.194713,Hauz Khas Village,28.554311,77.195140,Market,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop
533,Hauz Khas Village,28.553855,77.194713,Kunzum Travel Cafe,28.553684,77.194368,Art Gallery,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop
534,Hauz Khas Village,28.553855,77.194713,Imperfecto,28.554657,77.195092,Mediterranean Restaurant,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop
535,Hauz Khas Village,28.553855,77.194713,Naivedyam,28.554987,77.195104,South Indian Restaurant,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop
536,Hauz Khas Village,28.553855,77.194713,Coast Cafe,28.554779,77.195214,Café,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop
537,Hauz Khas Village,28.553855,77.194713,The Golconda Bowl,28.554972,77.194147,Indian Restaurant,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop
538,Hauz Khas Village,28.553855,77.194713,Chaayos,28.554247,77.194533,Tea Room,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop
539,Hauz Khas Village,28.553855,77.194713,Yeti - The Himalayan Kitchen,28.553656,77.194261,Tibetan Restaurant,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop
540,Hauz Khas Village,28.553855,77.194713,Social Offline,28.554461,77.193881,Lounge,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop
541,Hauz Khas Village,28.553855,77.194713,Gelato Roma,28.553314,77.194233,Ice Cream Shop,Indian Restaurant,Café,Bar,Restaurant,3,Italian Restaurant,Art Gallery,Lounge,Mediterranean Restaurant,Bakery,Coffee Shop


In [97]:
Delhi_merged[Delhi_merged['Cluster Label'] == 4]

,Neighborhood,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,Cluster Label,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
234,Connaught Place,28.631383,77.219792,Connaught Place | कनॉट प्लेस (Connaught Place),28.632731,77.220018,Plaza,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant
235,Connaught Place,28.631383,77.219792,Chaayos,28.631629,77.220364,Café,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant
236,Connaught Place,28.631383,77.219792,Starbucks,28.632011,77.217731,Coffee Shop,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant
237,Connaught Place,28.631383,77.219792,Farzi Cafe,28.632581,77.221125,Molecular Gastronomy Restaurant,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant
238,Connaught Place,28.631383,77.219792,Rajdhani Thali,28.629999,77.220401,Indian Restaurant,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant
239,Connaught Place,28.631383,77.219792,Fabindia,28.632012,77.217729,Clothing Store,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant
240,Connaught Place,28.631383,77.219792,Wenger's,28.633412,77.218292,Bakery,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant
241,Connaught Place,28.631383,77.219792,Johnny Rockets,28.630457,77.219594,Bistro,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant
242,Connaught Place,28.631383,77.219792,HOTEL SARAVANA BHAVAN,28.632319,77.216445,South Indian Restaurant,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant
243,Connaught Place,28.631383,77.219792,Barbeque Nation,28.630253,77.220985,BBQ Joint,Indian Restaurant,Café,Bar,Chinese Restaurant,4,Lounge,Coffee Shop,South Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Molecular Gastronomy Restaurant


# Clustering results:
**First Cluster**
The first cluster includes the areas of Vasant Vihar and Sriniwaspuri, with eloquent amenities, that is popular among the richer urban classes and has the best schools and offices.

**Second Cluster**
The second cluster includes the areas with bars and restaurants, popular among tourists and those looking for relaxation.

**Third Cluster**
The third cluster includes the areas of historical significance

**Fourth Cluster**
The fourth cluster includes the areas popular among the middle class and poorer people.

**Fifth Cluster**
The fifth cluster includes the areas which are predominantly industrial.